# Challenge 2: Sentiment Analysis

In this challenge we will learn about sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis refers to the process of *systematically identifying, extracting, quantifying, and studying affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy at the time of producing the piece of text. Why do we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what might have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide. 

"Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [ ]:
# import libraries for working with the dataset
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn import naive_bayes
import openpyxl
import regex as re


In [ ]:
# load data
tweetset = pd.read_csv(r'C:\Users\Ish\Documents\Ironhack Bootcamp\Unit 8\Day 2\L8.04\training.1600000.processed.noemoticon.csv', encoding = 'latin-1')

In [ ]:
# define functions for cleaning and preparing text data for analysis.
# preliminary cleaning
def clean_up(s):
    # sort through input and remove any patterns that look like urls.
    import regex as re
    # use regex to identify and drop URL strings.
    url_to_drop = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', s)
    try:
        s = s.replace(url_to_drop[0], " ")
    except:
        s=s
    # use regex to identify non-functional special characters
    chars_to_drop = re.findall('\W',s)
    for x in range(len(chars_to_drop)):
        s = s.replace(chars_to_drop[x],' ')
    # use regex to identify numerical values to drop.
    nums_to_drop = re.findall('\S?[0-9]', s)
    for n in range(len(nums_to_drop)):
        s = s.replace(nums_to_drop[n], ' ')
    # convert all remaining items to lower case
    return s.lower()

# tokenizing the cleaned text data
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

def tokenize(s):
    s_tokenized = nltk.word_tokenize(s)
    return s_tokenized

# produce stemmed and lemmatized results of the text input
def get_pos(x):
    tag = nltk.pos_tag([x])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist 

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
def stem_and_lemmatize(l):
    p_stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    l_stemmed = [p_stemmer.stem(l[x]) for x in range(len(l))]
    l_lemmatized =[lemmatizer.lemmatize(l[y], pos = (get_pos(l[y]))) for y in range(len(l))]
    return l_lemmatized

# remove stopwords from text sample.
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(l):
    l_stopped = [word for word in l if not word in stopwords.words()]
    return l_stopped

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [ ]:
# set columns headers to secribe the columns in the dataset
tweetset.columns = ['sentiment_label','tweet_id','post_timestamp','QUERY_status','username','user_tweet']

# apply preliminary cleaning function to dataset
tweetset['tweet_cleaned_up'] = tweetset['user_tweet'].apply(lambda x: clean_up(x))


In [ ]:
#apply tokenizer function to tweet columns.
tweetset['tweet_tokenized'] = tweetset['tweet_cleaned_up'].apply(lambda x: tokenize(x))

In [ ]:
tweetset.to_pickle('tweetset_unstemmed.pkl')

In [ ]:
# apply the lemmatizer function
tweetset['tweets_lemmatized'] = tweetset['tweet_tokenized'].apply(lambda x: stem_and_lemmatize(x))
tweetset['tweets_lemmatized'].head()

In [ ]:
# tweetset.to_pickle('tweetset_stopwordsin.pkl')

In [ ]:
# tweetset['tweet_text_cleaned'] = tweetset['tweets_lemmatized'].apply(lambda x: remove_stopwords(x))
# tweetset.head()

In [ ]:
tweetset = pickle.load(open('tweetset_stopwordsin.pkl', 'rb'))
tweetset.shape

In [ ]:
tweet_sample = tweetset.sample(frac = 0.01, replace = False)
tweet_sample.shape
tweet_sample.head()


In [ ]:
tweet_sample.columns

In [ ]:
tweet_sample['text_processed'] = tweet_sample['tweets_lemmatized'].apply(lambda x : remove_stopwords(x))
tweet_sample = tweet_sample[['sentiment_label', 'tweet_id', 'post_timestamp', 'QUERY_status',
       'username','text_processed']]
       


In [ ]:
tweet_sample.head()
tweet_sample.to_csv('tweets_sample_data.csv')

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [ ]:
# consolidate text data into a single corpus.
bag_of_text = []
for x in tweet_sample['text_processed']:
    for word in x:
        bag_of_text.append(word)

len(bag_of_text)


In [ ]:
# select 5000 most frequent words from bag_of_text
from nltk import FreqDist
frequent_dist = FreqDist(x.lower() for x in bag_of_text)
most_common5k = frequent_dist.most_common(5000)
most_common5k = [x[0] for x in most_common5k]


len(most_common5k)




### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# define function to take in a document (in this case a tokenized tweet) and construct the list
# of the tuple of feature dictionary and boolean value.

def get_features(document, feature_set):
    words = set(document)
    features = {}
    for w in feature_set:
        features[w] = (w in words)

    return features

document_features = [get_features(x,most_common5k) for x in tweet_sample['text_processed']]
document_categories =  [y for y in tweet_sample['sentiment_label']]


In [ ]:
feature_set = list(zip(document_features, document_categories))
feature_set


In [ ]:
len(feature_set)
#save features as a text file
feature_text = open('feature_text.txt', 'w+')
feature_text.write(str(document_features))
feature_text.close()


### Building and Training Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
from nltk import NaiveBayesClassifier
training_set = feature_set[:11999]
test_set = feature_set[12000:]
NBClassy = NaiveBayesClassifier.train(training_set)


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# test classifier and report the score
NB_accuracy = nltk.classify.accuracy(NBClassy, test_set)
print(f'Functional Accuracy Score = {round(NB_accuracy,2)}')

In [ ]:
# determine the best features
NBClassy.show_most_informative_features()


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here